In [1]:
import ee
ee.Authenticate(auth_mode='notebook')
ee.Initialize(project='global-booster-421311')


In [11]:
# Your catchments asset
catchments = ee.FeatureCollection('projects/global-booster-421311/assets/LakeCatchments_NA')
era = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR').filterDate('2019-01-01', '2020-01-01')
weeks = list(range(1, 54))

variables_mean = [
    'temperature_2m',
    'lake_mix_layer_temperature'
]
variables_sum = [
    'total_precipitation_sum',
    'surface_runoff_sum',
    'runoff_sum'
]

def features_for_week(week):
    week = ee.Number(week)
    start_day = week.subtract(1).multiply(7)
    end_day = week.multiply(7).subtract(1)
    start_date = ee.Date('2019-01-01').advance(start_day, 'day')
    end_date = ee.Date('2019-01-01').advance(end_day, 'day')

    week_imgs = era.filterDate(start_date, end_date.advance(1, 'day'))
    mean_img = week_imgs.mean().select(variables_mean)
    sum_img = week_imgs.sum().select(variables_sum)
    stack = mean_img.addBands(sum_img)
    
    stats = stack.reduceRegions(
        collection=catchments,
        reducer=ee.Reducer.mean().forEachBand(mean_img)
                .combine(ee.Reducer.sum().forEachBand(sum_img), '', False),
        scale=10000
    ).map(lambda f: f.set({
        'year': 2019,
        'week': week,
        'week_start': start_date.format('YYYY/MM/dd'),
        'week_end': end_date.format('YYYY/MM/dd')
    }))
    return stats

weekly_collections = ee.List(weeks).map(features_for_week)
all_features = ee.FeatureCollection(weekly_collections).flatten()

def drop_geometry(feature):
    return feature.setGeometry(None)
all_features_no_geom = all_features.map(drop_geometry)

export_fields = [
    'Lake_ID', 'year', 'week', 'week_start', 'week_end',
    'temperature_2m_mean',
    'lake_mix_layer_temperature_mean',
    'total_precipitation_sum_sum',
    'surface_runoff_sum_sum',
    'runoff_sum_sum'
]

task = ee.batch.Export.table.toDrive(
    collection=all_features_no_geom.select([
        'Lake_ID', 'year', 'week', 'week_start', 'week_end',
        'temperature_2m',
        'lake_mix_layer_temperature',
        'total_precipitation_sum',
        'surface_runoff_sum',
        'runoff_sum'
    ]),
    description='ERA5Land_Weekly_Catchments_2019_Climate',
    fileFormat='CSV'
)
task.start()
print("Export started! Monitor in GEE Tasks or Google Drive.")

Export started! Monitor in GEE Tasks or Google Drive.


In [ ]:
import pandas as pd

df = pd.read_csv('extracted_gee/ERA5Land_Weekly_Catchments_2019_Climate.csv')
import pandas as pd
# Drop unwanted columns if present
to_drop = ['.geo', 'system:index']
df = df.drop(columns=[col for col in to_drop if col in df.columns])

# Ensure the columns are ordered as desired
main_cols = ['Lake_ID', 'year', 'week', 'week_start', 'week_end']
other_cols = [col for col in df.columns if col not in main_cols]
df = df[main_cols + other_cols]

# Sort by Lake_ID and date (year, week)
df = df.sort_values(by=['Lake_ID', 'year', 'week'])

# Save cleaned table
df.to_csv('extracted_gee/ERA5Land_Weekly_Catchments_2019_Climate.csv', index=False)

print("✅ Done! Sorted, cleaned, and exported.")


✅ Done! Sorted, cleaned, and exported.
